In [ ]:
try:
    from pyscf import gto, dft
    print("Found pyscf")
except:
    print("Can't import, installing via pip")
    !pip3 install pyscf
    from pyscf import gto

try:
    import numpy as np
    print("Found numpy")
except:
    print("Can't import, installing via pip")
    !pip3 install numpy
    import numpy as np

try:
    import matplotlib.pyplot as plt
    print("Found matplotlib")
except:
    print("Can't import, installing via pip")
    !pip3 install matplotlib
    import matplotlib.pyplot as plt

try:
    from ase import Atoms
    from ase.build import molecule
    from ase.visualize import view
    from ase.io import write
    print("Found ase")
except:
    print("Can't import, installing via pip")
    !pip3 install ase
    from ase import Atoms    
    from ase.build import molecule
    from ase.visualize import view
    from ase.io import write

try:
    from pyscf.geomopt.geometric_solver import optimize
    print("Found geometry module")
except:
    print("Can't import, installing via pip")
    !pip3 install "pyscf[geomopt]"
    from pyscf.geomopt.geometric_solver import optimize

In [ ]:
def ase2pyscf():
    atomspyscf = []
    pos = atoms.get_positions()
    ele = atoms.get_chemical_symbols()
    numatoms = atoms.get_global_number_of_atoms()
    for atom in range(numatoms):
        elements = list(ele[atom])
        coords = np.ndarray.tolist(pos[atom])
        elencoord = elements + coords
        elencoordfix = ' '.join(map(str, elencoord))
        atomspyscf.append(elencoordfix)
    
    trueoutput = "; ".join(atomspyscf)
    
    return trueoutput

In [ ]:
moleculename = 'H2O'
atoms = molecule(moleculename)
inputatoms = ase2pyscf()

In [ ]:
mol = gto.Mole()
mol.build(atom = inputatoms,
          basis = 'ccpvdz', 
          spin = 0, 
          unit = 'angstrom')

#atom = inputatoms,
#atom = 'O 0.0 0.0 0.119262 ; H 0.0 0.763239 -0.477047 ; H 0.0 -0.763239 -0.477047',

In [ ]:
mf_hf = dft.RKS(mol)
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

In [ ]:
conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}

In [ ]:
dft_energies = []
def cb(envs):
  mf_hf = envs["g_scanner"].base
  dft_energies.append(mf_hf.e_tot)

In [ ]:
mol_eq = optimize(mf_hf, **conv_params, maxsteps=100, callback=cb)

In [ ]:
print(mol_eq.elements)
print(mol_eq.atom_coords())

In [ ]:
optmolecule = Atoms(mol_eq.elements, positions=mol_eq.atom_coords())

In [ ]:
view(optmolecule, viewer='x3d')

In [ ]:
print(dft_energies[-1])

In [ ]:
write(f'{round(dft_energies[-1] * 27.2114079527, 3)} -{moleculename}.xyz', optmolecule)

1. Consider the coordinates of the system. Write a function to calculate the bond distance of the O-H bond.
2. Compare this bond distance with the experimental bond distance for a gas phase molecule.
3. Write a function to calculate the bond angle of the H-O-H bond as well and also compare this with experiment.